<a href="https://colab.research.google.com/github/vidhya2324/Machine_learning/blob/main/Ethanol_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt

# Load the dataset

In [6]:
import zipfile
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt

# Unzip the uploaded file
zip_path = '/content/archive.zip'
extract_path = '/content/ethanol_data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check extracted files
extracted_files = os.listdir(extract_path)

# Initialize an empty list to hold the data
data_frames = []

# Read each Excel file and append to the list
for file in extracted_files:
    if file.endswith('.xlsx'):
        file_path = os.path.join(extract_path, file)
        df = pd.read_excel(file_path)
        data_frames.append(df)

In [7]:
# Concatenate all data frames
consolidated_data = pd.concat(data_frames, ignore_index=True)


In [8]:
# Display the first few rows of the consolidated data
print(consolidated_data.head())

              Estados  Cana-de-açúcar(mil toneladas)  Açúcar(mil toneladas)  \
0      Espírito Santo                       3938.757                 86.823   
1               Goiás                      21082.011                950.602   
2         Mato Grosso                      14928.015                536.234   
3  Mato Grosso do Sul                      14869.066                616.170   
4        Minas Gerais                      35723.246               2117.696   

   Etanol (mil m³)_Anidro  Etanol (mil m³)_Hidratado  Etanol (mil m³)_Total  \
0                 175.077                     77.384                252.461   
1                 463.888                    749.740               1213.628   
2                 382.134                    512.247                894.381   
3                 214.211                    662.562                876.773   
4                 575.223                   1199.765               1774.988   

   Cana-de-açúcar (mil toneladas)  Açúcar (mil ton

In [9]:
consolidated_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553 entries, 0 to 552
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Estados                         493 non-null    object 
 1   Cana-de-açúcar(mil toneladas)   464 non-null    float64
 2   Açúcar(mil toneladas)           464 non-null    float64
 3   Etanol (mil m³)_Anidro          493 non-null    float64
 4   Etanol (mil m³)_Hidratado       493 non-null    float64
 5   Etanol (mil m³)_Total           493 non-null    float64
 6   Cana-de-açúcar (mil toneladas)  29 non-null     float64
 7   Açúcar (mil toneladas)          29 non-null     float64
 8   ESTADO/SAFRA                    60 non-null     object 
 9   2002                            60 non-null     float64
 10  2003                            60 non-null     float64
 11  2004                            60 non-null     float64
 12  2005                            60 n

In [10]:
# Assuming the relevant column for ethanol demand is 'Etanol (mil m³)_Total'
ethanol_data = consolidated_data[['Etanol (mil m³)_Total']].dropna()

In [11]:
# Generate a date range assuming monthly data starting from 2002
date_range = pd.date_range(start='2002-01-01', periods=len(ethanol_data), freq='M')
ethanol_data['date'] = date_range
ethanol_data.set_index('date', inplace=True)

In [12]:
# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(ethanol_data.values)


In [13]:
# Create sequences
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)